In [1]:
import cv2
import glob
import os
import re # Import regex for parsing

# --- Parameters ---
# Folder where the cross-section images are saved
IMAGE_FOLDER = '.'

# Which zone index to compile (e.g., 0)
ZONE_TO_COMPILE = 0

# Desired output video name (will be saved *inside* the image folder)
VIDEO_NAME = os.path.join(IMAGE_FOLDER, f'zone_{ZONE_TO_COMPILE:04d}_timelapse.mp4')

# Frames per second for the output video
FPS = 3
# --- End Parameters ---

def get_timestep(filename):
    """Helper function to extract the timestep number for sorting."""
    try:
        # Assumes format like '..._TIMESTEP.png'
        # re.split finds numbers, take the last one
        parts = re.split(r'[_.]', filename)
        # parts will be ['zone', '0001', '0102', 'png']
        # We want the second to last part
        return int(parts[-2])
    except:
        return -1 # Error sorting

def create_video_from_images():
    # 1. Find all possible image files
    file_pattern = os.path.join(IMAGE_FOLDER, '*_*.png')
    all_files = glob.glob(file_pattern)

    # 2. Filter files to match the desired zone
    filtered_files = []
    for f in all_files:
        basename = os.path.basename(f)
        try:
            # Assumes format 'zone_INDEX_... .png'
            zone_part = basename.split('_')[0]
            if int(zone_part) == ZONE_TO_COMPILE:
                filtered_files.append(f)
        except Exception as e:
            print(f"Warning: Could not parse filename {basename}: {e}")
            
    files = filtered_files
    
    # 3. Sort the files numerically by timestep
    files.sort(key=get_timestep)

    if not files:
        print(f"Error: No image files found for zone {ZONE_TO_COMPILE} matching pattern: {file_pattern}")
        print("Please check if the 'cross_sections' folder exists and contains files.")
        return

    print(f"Found {len(files)} images for zone {ZONE_TO_COMPILE}. Compiling video...")
    # print("First file:", files[0]) # Uncomment for debugging
    # print("Last file:", files[-1])  # Uncomment for debugging

    # 4. Get image dimensions from the first file
    try:
        frame = cv2.imread(files[0])
        height, width, layers = frame.shape
        # OpenCV size is (width, height)
        size = (width, height)
    except Exception as e:
        print(f"Error reading first image ({files[0]}): {e}")
        return

    # 5. Initialize the VideoWriter object
    # We use the 'mp4v' codec, which is common for .mp4 files
    try:
        out = cv2.VideoWriter(VIDEO_NAME, cv2.VideoWriter_fourcc(*'mp4v'), FPS, size)
    except Exception as e:
        print(f"Error initializing VideoWriter (is OpenCV installed?): {e}")
        return

    # 6. Loop through all files, read them, and write to the video
    for filename in files:
        try:
            img = cv2.imread(filename)
            
            # --- FIX: Ensure frame dimensions match video writer dimensions ---
            # Check if image shape is different from the target size
            if img.shape[1] != size[0] or img.shape[0] != size[1]:
                # Resize the image to the video's size
                img = cv2.resize(img, size)
            # --- End Fix ---
                
            out.write(img)
        except Exception as e:
            print(f"Warning: Could not read or write frame {filename}: {e}")

    # 7. Release the video writer
    out.release()
    print(f"\nSuccessfully saved video: {VIDEO_NAME}")

if __name__ == "__main__":
    print("--- Starting Video Compilation ---")
    print(f"You may need to install OpenCV: pip install opencv-python")
    create_video_from_images()



--- Starting Video Compilation ---
You may need to install OpenCV: pip install opencv-python
Found 40 images for zone 0. Compiling video...

Successfully saved video: ./zone_0000_timelapse.mp4
